In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
import itertools
from statistical_functions import test_is_I1, test_is_tradable
import matplotlib.pyplot as plt

# Preprocess

In [45]:
data = pd.read_csv("ec_data.csv")
data.date = data.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
data['year'] = data.date.apply(lambda x: x.year)
data['week'] = data.date.apply(lambda x: x.dayofweek)
data['day'] = data.date.apply(lambda x: x.day)
data.head()

,code,name,industry_code,industry,date,price,year,week,day
0,1471,首利,M23D,電子零組件,2000-01-04,8.98,2000,1,4
1,2308,台達電,M23D,電子零組件,2000-01-04,25.32,2000,1,4
2,2327,國巨,M23D,電子零組件,2000-01-04,55.14,2000,1,4
3,2328,廣宇,M23D,電子零組件,2000-01-04,17.59,2000,1,4
4,2375,智寶,M23D,電子零組件,2000-01-04,46.39,2000,1,4


In [3]:
# create possible pairs
posible_pairs = list(
    itertools.combinations(data.code.unique(), 2)
)

In [4]:
posible_pairs[0]

(1471, 2308)

In [69]:
trade_timeframe = np.unique(list(zip(data.year, data.week)), axis=0)

In [72]:
temp_trade_timeframe = trade_timeframe[0]

In [79]:
test_data = data[(data.year == temp_trade_timeframe[0]) &
                 (data.week == temp_trade_timeframe[1])]

In [ ]:
test_is_tradable()